In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
def get_IPO_list_performance_data_from_38(page=1):
  total_data = []
  for i in range(1, 71):
    url = 'http://www.38.co.kr/html/fund/index.htm?o=nw&page=%s' % i
    response = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
    html = response.text
    soup = BeautifulSoup(html, 'lxml')
    data = soup.find('table', {'summary':'공모청약일정'})
    data = data.find_all('tr')[2:]
    total_data = total_data + data
  
  company_name = []
  new_list_date = []
  public_price = []
  initial_price = []
  initial_public = []
  firstday_close_price = []

  for row in range(0, len(total_data)):
    data_list = total_data[row].text.replace('\xa0', '').replace('\t\t', '').split('\n')[1:-1]
    if len(data_list) < 9:
      continue
    
    company_name.append(data_list[0].strip())
    new_list_date.append(data_list[1].strip())
    public_price.append(data_list[4].strip())
    initial_price.append(data_list[6].strip())
    initial_public.append(data_list[7].strip())
    firstday_close_price.append(data_list[8].strip())

  list_performance = pd.DataFrame({'기업명':company_name, 
                                   '신규상장일':new_list_date, 
                                   '공모가(원)':public_price, 
                                   '시초가(원)':initial_price, 
                                   '시초/공모(%)':initial_public, 
                                   '첫날종가(원)':firstday_close_price})
  
  list_performance['시초가(원)'] = list_performance['시초가(원)'].replace('-', np.nan)
  list_performance['시초/공모(%)'] = list_performance['시초/공모(%)'].replace('%', np.nan)
  list_performance['첫날종가(원)'] = list_performance['첫날종가(원)'].replace('예정', np.nan)
  
  return list_performance

if __name__ == '__main__':
  list_performance = get_IPO_list_performance_data_from_38()
  print(list_performance)

In [ ]:
## 데이터 저장

# import os
# directory = "/content/drive/MyDrive/공모가예측"
# file_path = os.path.join(directory, '공모가예측.csv')
# list_performance.to_csv(file_path, index=False)